In [1]:
# regular imports
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

%matplotlib inline

In [2]:
# display related imports
from IPython.display import display, Image, clear_output, HTML, IFrame

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# to save dataframe as an image
import dataframe_image as dfi

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Forest
from sklearn.tree import DecisionTreeClassifier

# Time Series
from datetime import datetime, timedelta
from dateutil.parser import parse
from pandas.tseries.offsets import DateOffset

# random numbers
import random

In [3]:
df = pd.read_csv('data/df_for_filter.csv')
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
print(df.head())
print(df.shape, 'should be (2096800, 126)') 

   id  loan_amnt  funded_amnt  funded_amnt_inv  int_rate  installment grade  \
0   0     3600.0       3600.0           3600.0     13.99       123.03     C   
1   1    24700.0      24700.0          24700.0     11.99       820.28     C   
2   2    20000.0      20000.0          20000.0     10.78       432.66     B   
3   3    35000.0      35000.0          35000.0     14.85       829.90     C   
4   4    10400.0      10400.0          10400.0     22.45       289.91     F   

  sub_grade emp_length home_ownership  ...  debt_settlement_flag_date  \
0        C4  10+ years       MORTGAGE  ...                        NaN   
1        C1  10+ years       MORTGAGE  ...                        NaN   
2        B4  10+ years       MORTGAGE  ...                        NaN   
3        C5  10+ years       MORTGAGE  ...                        NaN   
4        F1    3 years       MORTGAGE  ...                        NaN   

  issue_year issue_month fico term_60  last_pay_d  months_of_pay  loss_amnt  \
0      

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [5]:
# initual setup - setting up cash account
df_cash = []
df_=[]
def cash_init(start_date='20070601', start_=200):
    data = {'Date': [start_date],
           'from': ['Start'],
            'cash': [start_],
            'invested': [0] 
           }
    df_cash = pd.DataFrame(data, columns = ['Date', 'from', 'cash', 'invested'])
    df_cash['Date'] = pd.to_datetime(df_cash['Date']) 
    return df_cash


def loan_picker(issue_month_, issue_year_):
    df_filtered = df[(df['issue_year']==issue_year_) & 
       (df['grade'].isin(grade_)) & 
       (df['fico']>fico_) & 
       (df['int_rate']>min_int_rate_) & 
       (df['home_ownership']!='RENT') & 
       (df['emp_length']!= emp_length_) &
       (df['annual_inc'] > annual_inc_) &
       (df['earliest_credit_10_more'] == earliest_credit_10_more_) &
       (df['loan_amnt'] < loan_amnt_) &
       (df['term_60']==term_60_) &
       (df['purpose'].isin(purpose_)) &
       (df['total_acc'] < total_acc_) &
       (df['installment'] < installment_) &
       (df['issue_month']==issue_month_) & #.isin(issue_month_)&
       (df['inq_last_6mths']==inq_last_6mths_)
      ].reset_index(drop=True)
    return df_filtered['id']


def loan_calc(inv, df_):
    '''inv = investement amount 
    df_ is the dataframe
    this function calculates monthly payments and amount returned based on a loan and amount invested
    '''
    #rec_back = 0
    I_ = []
    P_ = []
    inv_ = []
    LAST_PAYMENT = round(df_['last_pymnt_amnt'] * (inv/df_['funded_amnt_inv']), 2)
    TERM = 36 if df_['term_60']==0 else 60
    END_MNTH = min([TERM, df_['months_of_pay']+1])
    R = 1 + (df_['int_rate'])/(12*100)              # calc monthly rate
    X = inv * (R**TERM)*(1-R)/(1-R**TERM)           # calc monthly payment
    I_.append(0)
    P_.append(0)
    inv_.append(inv)
    for n_ in range(1,END_MNTH + 1):                 # goes through months, skips first month (first payment is the following month)
        I = round(inv * (R-1), 2)            # interest calculation
        if n_ == END_MNTH:                           # if last month (last month is lessor of term or months Pay field)
            X = LAST_PAYMENT                        # if last payment, get last payment from dataframe
            I = round((df_['out_prncp'] - (inv - X)), 2)
        inv = round(inv - (X-I),2)           # current invested amount is previous invested minus (Payment - interest)    
        #print((X - I, I))
        I_.append(I)
        P_.append(X - I)
        inv_.append(I - X)
    return (P_, I_, inv_, df_)

def add_cash(df_cash, df_inv):
    return df_cash.append(df_inv, ignore_index=True)


def invest(df_cash_=df_cash, loan_='loan_x',df_=df):       
    df_investment = pd.DataFrame(columns=['Date','from', 'P', 'I', 'cash', 'invested'])
    df_investment['P'] = P
    df_investment['I'] = I
    df_investment['invested'] = inv
    df_investment['Date'] = [parse(df_['issue_d']) + DateOffset(months=i) for i in range(0, df_investment['P'].shape[0])]
    df_investment['cash'] = df_investment['P'] + df_investment['I']
    df_investment['from'] = loan_
    df_investment.loc[0,'cash'] = df_investment.loc[0, 'invested']*-1
    print(df_investment)
    return add_cash(df_cash_, df_investment)


def run_loan(id_, fields_, invest_):
    df_ = df.loc[id_, fields_]
    #rec_back = 0
    return loan_calc(invest_, df_)

#gets cash at a date
def get_cash(date_):
    return round(df_cash.loc[df_cash['Date']==date_]['cash'].sum(),2)

In [6]:
fields_ = ['id',
           'loan_amnt',
           'funded_amnt',
           'funded_amnt_inv',
           'int_rate',
           'installment',
           'grade',
           'sub_grade',
           'issue_d',
           'out_prncp',
           'out_prncp_inv',
           'total_pymnt',
           'total_pymnt_inv',
           'total_rec_prncp',
           'total_rec_int',
           'total_rec_late_fee',
           'recoveries',
           'collection_recovery_fee',
           'last_pymnt_d',
           'last_pymnt_amnt',
           'months_of_pay',
           'good',
           'term_60'
          ]
set(fields_) - set(df.columns) 

set()

In [7]:
start_date = '20071201'
issue_year_ = 2012
grade_ = ['D', 'E', 'F']
fico_ = 1 # above this
min_int_rate_ = 13.5
emp_length_ = '< 1 year'
annual_inc_ = 70000
earliest_credit_10_more_ = 1
loan_amnt_ = 25001
term_60_ = 0
chargeoff_within_12_mths_ = 0
purpose_ = ['credit_card', 'debt_consolidation']
total_acc_ = 25 # less than this amount
installment_ = 800
#issue_month_ = [7]
inq_last_6mths_ = 0


df_cash = cash_init(start_date, 250)
loans_=[]
random.seed(22)
years = [2008, 2009]#, 2010, 2011, 2012, 2013]
for y in years: # selecting years in list
    for m in range (1,13): # months in year
        loans_ = (loan_picker(m, y)) # gets loan numbers that match criteria
        # print(m, y, len(loans_))
        old_cash = get_cash(str(m)+'/'+str(y)) # gets cash that has been returned
        print('X'*100)
        if len(loans_) > 5: # need to change this $50 per loan333
            loans_ = pd.Series(random.sample(list(loans_), 5))
        print(loans_)
        for loan_ in loans_:
            P, I, inv, df_ = run_loan(loan_, fields_, (df_cash.loc[0, 'cash'] + old_cash)/len(loans_))
            df_cash = invest(df_cash, loan_, df_)
            return_ = df_cash[df_cash['from']==loan_]['cash'] 
            print(f'NPV  :  {np.npv(.13/12, return_) :>6.2f}')
            print(f'IRR  :  {np.irr(return_*-1)* 12:>7.2%}')

1 2008 0
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Series([], Name: id, dtype: int64)
2 2008 0
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Series([], Name: id, dtype: int64)
3 2008 1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
0    1650866
Name: id, dtype: int64
         Date     from         P      I        cash    invested
0  2008-03-01  1650866  0.000000   0.00 -250.000000  250.000000
1  2008-04-01  1650866  5.692417   2.76    8.452417   -5.692417
2  2008-05-01  1650866  5.752417   2.70    8.452417   -5.752417
3  2008-06-01  1650866  5.822417   2.63    8.452417   -5.822417
4  2008-07-01  1650866  5.882417   2.57    8.452417   -5.882417
5  2008-08-01  1650866  5.952417   2.50    8.452417   -5.952417
6  2008-09-01  1650866  6.012417   2.44    8.452417   -6.012417
7  2008-10-01  1650866  6.082417   2.37    8.452417 

2 2009 2
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
0    1649404
1    1649481
Name: id, dtype: int64
        Date     from           P      I        cash    invested
0 2009-02-01  1649404    0.000000   0.00 -137.985000  137.985000
1 2009-03-01  1649404    3.423608   0.88    4.303608   -3.423608
2 2009-04-01  1649404    3.443608   0.86    4.303608   -3.443608
3 2009-05-01  1649404    3.463608   0.84    4.303608   -3.463608
4 2009-06-01  1649404    3.483608   0.82    4.303608   -3.483608
5 2009-07-01  1649404    3.513608   0.79    4.303608   -3.513608
6 2009-08-01  1649404  120.670000  24.88  145.550000 -120.670000
NPV  :   19.29
IRR  :   41.76%
         Date     from         P     I        cash    invested
0  2009-02-01  1649481  0.000000  0.00 -137.985000  137.985000
1  2009-03-01  1649481  3.103376  1.62    4.723376   -3.103376
2  2009-04-01  1649481  3.133376  1.59    4.723376   -3.133376
3  2009-05-01  1649481  3.173376  1.55

10 2009 1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
0    1646725
Name: id, dtype: int64
         Date     from          P     I        cash    invested
0  2009-10-01  1646725   0.000000  0.00 -310.310000  310.310000
1  2009-11-01  1646725   6.917828  3.78   10.697828   -6.917828
2  2009-12-01  1646725   7.007828  3.69   10.697828   -7.007828
3  2010-01-01  1646725   7.087828  3.61   10.697828   -7.087828
4  2010-02-01  1646725   7.177828  3.52   10.697828   -7.177828
5  2010-03-01  1646725   7.267828  3.43   10.697828   -7.267828
6  2010-04-01  1646725   7.347828  3.35   10.697828   -7.347828
7  2010-05-01  1646725   7.437828  3.26   10.697828   -7.437828
8  2010-06-01  1646725   7.527828  3.17   10.697828   -7.527828
9  2010-07-01  1646725   7.627828  3.07   10.697828   -7.627828
10 2010-08-01  1646725   7.717828  2.98   10.697828   -7.717828
11 2010-09-01  1646725   7.807828  2.89   10.697828   -7.807828
12 2010-10-01  164672

In [8]:
print(years)
no_y = len(years)
print(no_y*12*250)
print(df_cash['cash'].sum())
print(df_cash['cash'].sum()+(no_y*12*250))

[2008, 2009]
6000
648.1106792938809
6648.110679293881


In [9]:
np.fv(.14/12, 12*no_y, -250, 0, )
# 0, 13.41, 29.6, 11.39, 0.73

6878.2950021862935

In [10]:
sp_500_return = [-38.49, 23.45, 12.78, 0, 13.41, 29.6]

np.mean(sp_500_return)#, 11.39, .73])

6.791666666666667

In [11]:
df_cash.groupby('from').count()

,Date,cash,invested,P,I
from,,,,,
1645560,19,19,19,19,19
1646078,9,9,9,9,9
1646725,37,37,37,37,37
1647125,37,37,37,37,37
1647762,37,37,37,37,37
1649074,5,5,5,5,5
1649404,7,7,7,7,7
1649481,36,36,36,36,36
1649828,22,22,22,22,22


In [12]:
# A look at loans and cash
temp_cash_df = df_cash.groupby('from', as_index = False).agg({'Date': ['first', 'last'], 'cash': ['sum', 'first', 'last']})
temp_cash_df['Date', 'diff']= round((temp_cash_df['Date', 'last'] - temp_cash_df['Date', 'first']) / np.timedelta64(1,'M'),0)
temp_cash_df = temp_cash_df[:-1]
temp_cash_df

from       Date                   cash                   Date
                 first       last         sum    first    last  diff
0   1645560 2009-12-01 2011-06-01  -63.804807 -165.820    5.78  18.0
1   1646078 2009-11-01 2010-07-01  -16.607441 -321.010  229.95   8.0
2   1646725 2009-10-01 2012-10-01   75.323988 -310.310   11.21  36.0
3   1647125 2009-09-01 2012-09-01   76.173538 -299.870   10.62  36.0
4   1647762 2009-08-01 2012-08-01  133.760959 -560.110   22.81  36.0
5   1649074 2009-03-01 2009-07-01   18.594068 -285.000  275.11   4.0
6   1649404 2009-02-01 2009-08-01   29.083038 -137.985  145.55   6.0
7   1649481 2009-02-01 2012-01-01   30.939770 -137.985    8.33  35.0
8   1649828 2008-12-01 2010-09-01  -52.512146 -267.190   39.07  21.0
9   1650866 2008-03-01 2011-03-01   69.174588 -250.000   23.34  36.0
10  1652715 2009-11-01 2012-11-01   43.749437 -165.820    6.53  36.0
11  1653631 2008-04-01 2011-03-01   54.235688 -258.450   15.59  35.0

In [13]:
temp_cash_df['cash','sum'].sum()

398.1106792938807

In [14]:
temp_cash_df

from       Date                   cash                   Date
                 first       last         sum    first    last  diff
0   1645560 2009-12-01 2011-06-01  -63.804807 -165.820    5.78  18.0
1   1646078 2009-11-01 2010-07-01  -16.607441 -321.010  229.95   8.0
2   1646725 2009-10-01 2012-10-01   75.323988 -310.310   11.21  36.0
3   1647125 2009-09-01 2012-09-01   76.173538 -299.870   10.62  36.0
4   1647762 2009-08-01 2012-08-01  133.760959 -560.110   22.81  36.0
5   1649074 2009-03-01 2009-07-01   18.594068 -285.000  275.11   4.0
6   1649404 2009-02-01 2009-08-01   29.083038 -137.985  145.55   6.0
7   1649481 2009-02-01 2012-01-01   30.939770 -137.985    8.33  35.0
8   1649828 2008-12-01 2010-09-01  -52.512146 -267.190   39.07  21.0
9   1650866 2008-03-01 2011-03-01   69.174588 -250.000   23.34  36.0
10  1652715 2009-11-01 2012-11-01   43.749437 -165.820    6.53  36.0
11  1653631 2008-04-01 2011-03-01   54.235688 -258.450   15.59  35.0

In [25]:
df_cash.groupby('from').agg({'Date': 'first', 'cash': 'first'})[:-1].sort_values(by='Date')#.sum()

,Date,cash
from,,
1650866,2008-03-01,-250.000
1653631,2008-04-01,-258.450
1649828,2008-12-01,-267.190
1649404,2009-02-01,-137.985
1649481,2009-02-01,-137.985
1649074,2009-03-01,-285.000
1647762,2009-08-01,-560.110
1647125,2009-09-01,-299.870
1646725,2009-10-01,-310.310


In [26]:
9*250

2250

In [138]:
df_cash.groupby('from').agg({'Date': 'first', 'cash': 'first'})[:-1].sum()

cash   -3159.55
dtype: float64

In [139]:
df_cash.groupby('from').agg({'Date': 'first', 'cash': 'first'})[:-1].sum()+(10*250)

cash   -659.55
dtype: float64

In [133]:
for loan_ in df_cash.groupby('from')['from'].first():
    print(df[df['id']==loan_][['good','months_of_pay']])

         good  months_of_pay
1645540  True             35
         good  months_of_pay
1646058  True             33
         good  months_of_pay
1646705  True             35
          good  months_of_pay
1647105  False             12
         good  months_of_pay
1647742  True             25
         good  months_of_pay
1649054  True             36
         good  months_of_pay
1649384  True             35
         good  months_of_pay
1649461  True             35
         good  months_of_pay
1649808  True             28
         good  months_of_pay
1650846  True             35
         good  months_of_pay
1652694  True              8
          good  months_of_pay
1653610  False              9
Empty DataFrame
Columns: [good, months_of_pay]
Index: []


In [132]:
df[df['id'] == 1649808][['loan_amnt', 'good', 'months_of_pay']]

,loan_amnt,good,months_of_pay
1649788,15000.0,True,2


In [105]:
print(df_cash.groupby('from')['from']
     .first()
     .apply(
         lambda x: 
             df[df['id'] == x ]
               [['good','months_of_pay']])
)



    

from
1645560             good  months_of_pay
1645540  True    ...
1646078             good  months_of_pay
1646058  True    ...
1646725             good  months_of_pay
1646705  True    ...
1647125              good  months_of_pay
1647105  False  ...
1647762             good  months_of_pay
1647742  True    ...
1649074             good  months_of_pay
1649054  True    ...
1649404             good  months_of_pay
1649384  True    ...
1649481             good  months_of_pay
1649461  True    ...
1649828             good  months_of_pay
1649808  True    ...
1650866             good  months_of_pay
1650846  True    ...
1652715             good  months_of_pay
1652694  True    ...
1653631              good  months_of_pay
1653610  False  ...
Start      Empty DataFrame
Columns: [good, months_of_pay]...
Name: from, dtype: object


In [37]:
#
#print(return_[1:37])
print(f'NPV  :  {np.npv(.13/12, return_) :>6.2f}')
print(f'IRR  :  {np.irr(return_*-1)* 12:>7.2%}')
print(f"MIRR :  {np.mirr(return_*-1, 4/12, 13/12)*.12:>7.2%}")

NPV  :    6.85
IRR  :   15.85%
MIRR :   14.62%


In [38]:
df_ = df.loc[1307326, fields_]
#   1307326
invested = 700
#rec_back = 0
P, I, inv = loan_calc(invested, df_)
df_cash = invest()

ValueError: too many values to unpack (expected 3)

In [153]:
print(df_cash['invested'].sum())
df_cash.shape

0.005119356926684304


(51, 6)

In [154]:
df_cash

,Date,from,cash,invested,P,I
0,2007-01,starting,10000.000000,0.000000,NaN,NaN
1,2015-12-01 00:00:00,loan_x,-700.000000,700.000000,0.000000,0.00
2,2016-01-01 00:00:00,loan_x,22.058374,-17.168374,17.168374,4.89
3,2016-02-01 00:00:00,loan_x,22.058374,-17.288374,17.288374,4.77
4,2016-03-01 00:00:00,loan_x,22.058374,-17.408374,17.408374,4.65
...,...,...,...,...,...,...
46,2014-12-01 00:00:00,loan_x,23.414181,-17.524181,17.524181,5.89
47,2015-01-01 00:00:00,loan_x,23.414181,-17.704181,17.704181,5.71
48,2015-02-01 00:00:00,loan_x,23.414181,-17.884181,17.884181,5.53
49,2015-03-01 00:00:00,loan_x,23.414181,-18.074181,18.074181,5.34


In [155]:
print(df_cash['cash'])
print(df_cash['cash'].sum())
print(df_cash.shape)
print(df_cash[df_cash['from']=='loan_x']['cash'])
return_ = df_cash[df_cash['from']=='loan_x']['cash'] #-df_cash['I'][1:37]
#
#print(return_[1:37])
print(f'NPV  :  {np.npv(.13/12, return_) :>6.2f}')
print(f'IRR  :  {np.irr(return_*-1)* 12:>7.2%}')
print(f"MIRR :  {np.mirr(return_*-1, 4/12, 13/12)*.12:>7.2%}")



0     10000.000000
1      -700.000000
2        22.058374
3        22.058374
4        22.058374
          ...     
46       23.414181
47       23.414181
48       23.414181
49       23.414181
50      522.860000
Name: cash, Length: 51, dtype: float64
10189.904880643076
(51, 6)
1    -700.000000
2      22.058374
3      22.058374
4      22.058374
5      22.058374
         ...    
46     23.414181
47     23.414181
48     23.414181
49     23.414181
50    522.860000
Name: cash, Length: 50, dtype: float64
NPV  :  -39.37
IRR  :   10.28%
MIRR :   14.23%


In [318]:
for i,v in enumerate(df.columns):
    print(i,"'"+v+"'")

0 'id'
1 'loan_amnt'
2 'funded_amnt'
3 'funded_amnt_inv'
4 'int_rate'
5 'installment'
6 'grade'
7 'sub_grade'
8 'emp_length'
9 'home_ownership'
10 'annual_inc'
11 'verification_status'
12 'issue_d'
13 'loan_status'
14 'purpose'
15 'addr_state'
16 'dti'
17 'delinq_2yrs'
18 'inq_last_6mths'
19 'mths_since_last_delinq'
20 'mths_since_last_record'
21 'open_acc'
22 'pub_rec'
23 'revol_bal'
24 'revol_util'
25 'total_acc'
26 'initial_list_status'
27 'out_prncp'
28 'out_prncp_inv'
29 'total_pymnt'
30 'total_pymnt_inv'
31 'total_rec_prncp'
32 'total_rec_int'
33 'total_rec_late_fee'
34 'recoveries'
35 'collection_recovery_fee'
36 'last_pymnt_d'
37 'last_pymnt_amnt'
38 'last_credit_pull_d'
39 'last_fico_range_high'
40 'last_fico_range_low'
41 'collections_12_mths_ex_med'
42 'mths_since_last_major_derog'
43 'policy_code'
44 'application_type'
45 'annual_inc_joint'
46 'dti_joint'
47 'verification_status_joint'
48 'acc_now_delinq'
49 'tot_coll_amt'
50 'tot_cur_bal'
51 'open_acc_6m'
52 'open_act_il'


def loan_picker(issue_month_):
    start_date = '20120101'
    issue_year_ = 2014
    grade_ = ['C','D', 'E', 'F', 'G']
    fico_ = 1 # above this
    min_int_rate_ = 10.5
    emp_length_ = '< 1 year'
    annual_inc_ = 50000
    earliest_credit_10_more_ = 1
    loan_amnt_ = 25001
    term_60_ = 0
    chargeoff_within_12_mths_ = 0
    purpose_ = ['credit_card', 'debt_consolidation']
    total_acc_ = 25 # less than this amount
    installment_ = 800
    #issue_month_ = [7]
    inq_last_6mths_ = 0
    df_filtered = df[(df['issue_year']==issue_year_) & 
       (df['grade'].isin(grade_)) & 
       (df['fico']>fico_) & 
       (df['int_rate']>min_int_rate_) & 
       (df['home_ownership']!='RENT') & 
       (df['emp_length']!= emp_length_) &
       (df['annual_inc'] > annual_inc_) &
       (df['earliest_credit_10_more'] == earliest_credit_10_more_) &
       (df['loan_amnt'] < loan_amnt_) &
       (df['term_60']==term_60_) &
       (df['purpose'].isin(purpose_)) &
       (df['total_acc'] < total_acc_) &
       (df['installment'] < installment_) &
       (df['issue_month'].isin([3,4,5])&
       (df['inq_last_6mths']==inq_last_6mths_)
       )
      ].reset_index(drop=True)
    print(df_filtered['id'])
    return df_filtered['id']


start_date = '20120101'
issue_year_ = 2014
grade_ = ['D', 'E', 'F', 'G']
fico_ = 1 # above this
min_int_rate_ = 13.5
emp_length_ = '< 1 year'
annual_inc_ = 70000
earliest_credit_10_more_ = 1
loan_amnt_ = 25001
term_60_ = 0
chargeoff_within_12_mths_ = 0
purpose_ = ['credit_card', 'debt_consolidation']
total_acc_ = 25 # less than this amount
installment_ = 800
#issue_month_ = [7]
inq_last_6mths_ = 0


df_cash = cash_init(start_date, 650)
loans_=[]
for m in range (1,13):
    loans_.append(loan_picker(m))
    print(m)
    print(loans_)
print(loans_)

In [92]:
df_filtered = df[(df['issue_year']==issue_year_) & 
   (df['grade'].isin(grade_)) & 
   (df['fico']>fico_) & 
   (df['int_rate']>min_int_rate_) & 
   (df['home_ownership']!='RENT') & 
   (df['emp_length']!= emp_length_) &
   (df['annual_inc'] > annual_inc_) &
   (df['earliest_credit_10_more'] == earliest_credit_10_more_) &
   (df['loan_amnt'] < loan_amnt_) &
   (df['term_60']==term_60_) &
   (df['purpose'].isin(purpose_)) &
   (df['total_acc'] < total_acc_) &
   (df['installment'] < installment_) &
   (df['issue_month'].isin(issue_month_)&
   (df['inq_last_6mths']==inq_last_6mths_)
   )
  ].reset_index(drop=True)

print(df_filtered.groupby('good')['loss_amnt'].count())
loss = round(df_filtered[df_filtered['good']!=True]['loss_amnt'].sum(),-1)
loan = round(df_filtered[df_filtered['good']!=True]['loan_amnt'].sum(),-1)
print('Loss amount :', loss)
print('Loan amount :', loan)
print('Loss / Loan :', round(loss/loan, 3))
# total_pymnt_inv loan_amnt
rec = df_filtered['total_pymnt_inv'].sum()
paid_out = df_filtered['loan_amnt'].sum()
gain = rec - paid_out
print('Total rec   :', round(rec, -1))
print('Total paid  :', round(paid_out, -1))
print('Total gain  :', round(gain, -1))
print('Percent     :', round(gain/paid_out * 100, 1))


df_filtered

good
True    8
Name: loss_amnt, dtype: int64
Loss amount : 0.0
Loan amount : 0.0
Loss / Loan : nan
Total rec   : 134730.0
Total paid  : 109800.0
Total gain  : 24930.0
Percent     : 22.7


,id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,debt_settlement_flag_date,issue_year,issue_month,fico,term_60,last_pay_d,months_of_pay,loss_amnt,good,earliest_credit_10_more
0,1934941,16000.0,16000.0,16000.0,19.99,594.54,E,E1,10+ years,MORTGAGE,79000.0,Verified,2012-02-01,Fully Paid,debt_consolidation,WA,23.89,0.0,0.0,47.0,NaN,9.0,0.0,11187.0,90.2,17.0,f,0.0,0.0,18616.271609,18616.27,16000.0,2616.27,0.0,0.0,0.0,Jan-2013,12681.67,Feb-2013,689.0,685.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,NaN,2012,2,2,0,2013-01-01,10,0.0,True,1
1,1935816,21000.0,21000.0,21000.0,17.58,754.79,D,D4,2 years,MORTGAGE,72000.0,Verified,2012-02-01,Fully Paid,debt_consolidation,MA,11.63,2.0,0.0,12.0,NaN,11.0,0.0,19116.0,80.7,21.0,f,0.0,0.0,27172.120013,27172.12,21000.0,6172.12,0.0,0.0,0.0,Feb-2015,777.08,Mar-2019,769.0,765.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,NaN,2012,2,2,0,2015-02-01,35,0.0,True,1
2,1937707,20000.0,20000.0,20000.0,16.29,706.01,D,D1,6 years,MORTGAGE,92000.0,Verified,2012-01-01,Fully Paid,debt_consolidation,NC,8.80,0.0,0.0,NaN,NaN,10.0,0.0,19212.0,95.9,11.0,f,0.0,0.0,25416.250000,25416.25,20000.0,5416.25,0.0,0.0,0.0,Jan-2015,718.62,Jan-2015,694.0,690.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,NaN,2012,1,3,0,2015-01-01,35,0.0,True,1
3,1937923,8000.0,8000.0,8000.0,17.58,287.54,D,D4,10+ years,MORTGAGE,75000.0,Not Verified,2012-01-01,Fully Paid,debt_consolidation,NJ,11.25,0.0,0.0,48.0,NaN,13.0,0.0,951

In [127]:
df_cash = cash_init(start_date, 300)
print(len(loans_))
print(df_cash.loc[0, 'cash'])
for loan_ in loans_:
    P, I, inv, df_ = run_loan(loan_, fields_, df_cash.loc[0, 'cash']/len(loans_))
    df_cash = invest(df_cash, loan_, df_)
    return_ = df_cash[df_cash['from']==loan_]['cash'] 
    print(f'NPV  :  {np.npv(.13/12, return_) :>6.2f}')
    print(f'IRR  :  {np.irr(return_*-1)* 12:>7.2%}')
    #print(f"MIRR :  {np.mirr(return_*-1, 4/12, 13/12)*.12:>7.2%}")

1
300
         Date     from          P     I        cash    invested
0  2012-12-01  1888103   0.000000  0.00 -300.000000  300.000000
1  2013-01-01  1888103   3.992957  2.22    6.212957   -3.992957
2  2013-02-01  1888103   4.012957  2.20    6.212957   -4.012957
3  2013-03-01  1888103   4.042957  2.17    6.212957   -4.042957
4  2013-04-01  1888103   4.072957  2.14    6.212957   -4.072957
..        ...      ...        ...   ...         ...         ...
45 2016-09-01  1888103   5.522957  0.69    6.212957   -5.522957
46 2016-10-01  1888103   5.562957  0.65    6.212957   -5.562957
47 2016-11-01  1888103   5.602957  0.61    6.212957   -5.602957
48 2016-12-01  1888103   5.642957  0.57    6.212957   -5.642957
49 2017-01-01  1888103  71.230000  0.63   71.860000  -71.230000

[50 rows x 6 columns]
NPV  :  -26.03
IRR  :    8.93%


In [50]:
df_cash.groupby('Date').sum()

,cash,invested,P,I
Date,,,,
2009-12-01,300.000000,0.000000,0.000000,0.00
2010-12-01,-300.000000,300.000000,0.000000,0.00
2011-01-01,10.376700,-6.666700,6.666700,3.71
2011-02-01,10.376700,-6.746700,6.746700,3.63
2011-03-01,10.376700,-6.836700,6.836700,3.54
2011-04-01,10.376700,-6.916700,6.916700,3.46
2011-05-01,10.376700,-7.006700,7.006700,3.37
2011-06-01,10.376700,-7.096700,7.096700,3.28
2011-07-01,10.376700,-7.186700,7.186700,3.19
